In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import os
import gc
import re
from datetime import datetime
# 讀取 Google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
pd.__version__

'0.25.3'

# add dict Simplified to traditional

In [3]:
path = r'G:\py_workspace\20191103_ptt_public_opinion\data\id_check\\'
#os.listdir(path + 'source')

In [4]:
source = pd.read_csv(path + 'source/gephi_trns_v2.csv')
usr_lst = ['yuxds', 'omanorbo', 'kieranc', 'mark2165']
usr_lst.extend(source.sort_values('pagerank', ascending=False).id.to_list())

In [5]:
source.sort_values('pagerank', ascending=False)

,Unnamed: 0,community,id,pagerank
30771,0,4,Neptunium,254.015470
5012,0,0,kero2377,213.156853
21324,0,3,Wojnarowski,210.151955
6121,0,0,nicholas0406,144.894421
41340,0,4,Kevin1103,141.652942
...,...,...,...,...
16957,0,1,xxxdog,0.150000
2611,0,0,captainmm,0.150000
2610,0,0,capsspac,0.150000
2609,0,0,capdog,0.150000


In [6]:
source.community.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [7]:
source.dtypes

Unnamed: 0      int64
community       int64
id             object
pagerank      float64
dtype: object

In [8]:
usr_lst[:20]

['yuxds',
 'omanorbo',
 'kieranc',
 'mark2165',
 'Neptunium',
 'kero2377',
 'Wojnarowski',
 'nicholas0406',
 'Kevin1103',
 'diefishfish',
 'KingKingCold',
 'hong888',
 'papamama007',
 'jobli',
 'ray90910',
 'ben108472',
 'ice80712',
 'AskWhat',
 'jason748',
 'oftisa']

In [9]:
#usr_lst = ['yuxds', 'omanorbo', 'kieranc', 'mark2165']
#usr_lst.extend(source.id.to_list())

# parser function

In [9]:
def content_parser(soup):
    cont_dict = {}
    url_cnt = 0
    art_id = 'invalid'
    ip = 'invalid'
    ori_url_split = 'invalid'
    
    temp_title = soup.select('.thread-title')[0].text
        
    #content parsing
    temp_content = soup.select('#main-content')

    if temp_content:
        t1 = temp_content[0]
    else:
        return 'NoContent'
    
    for x in soup.select('span.f2'):
        if x.text.startswith('※ 文章網址: https://www.ptt.cc/bbs'):
            ori_url_split = x.a['href'].split('/')
            ip = x.previous_element.split(':')[-1].split()[0]
            url_cnt = url_cnt + 1
        elif x.text.startswith('※ 文章網址:'):
            pre_ip = [x for x in x.previous_elements][1]
            pre_url = x.next_sibling
            
            if type(pre_url) is bs4.element.NavigableString:
                #old article
                ori_url_split = x.a['href'].split('/')
            else:
                ori_url_split = x.next_sibling.text.split('/')
                
            if type(pre_ip) is bs4.element.NavigableString:
                ip = pre_ip.split(':')[-1].split()[0]
            else:
                #old article
                ip = pre_ip.text.split(':')[-1].split()[0]
            url_cnt = url_cnt + 1            
               
        if url_cnt > 1:
            ori_url_split = 'invalid'
            ip = 'invalid'
            break
            #raise ValueError('More than one url....')
    
    if ori_url_split != 'invalid':
        art_id = ori_url_split[-1].strip('.html')
        #board = ori_url_split[-2]
    for tag in remove_tags:
        if len(tag) == 1:
            iter_tags = t1.find_all(tag[0])
        else:
            iter_tags = t1.find_all(tag[0], tag[1])
        
        for x in iter_tags:
            x.decompose()
        
    cont_dict.update({
          'article_id' : art_id
        , 'article_title' : temp_title 
        , 'board' : soup.select('.board')[0].text.strip()
        , 'content' : t1.text.replace('\n', ' ')
        , 'date' : soup.select('.post-time')[0].text.strip()
        , 'ip' : ip
    })
    return cont_dict

In [10]:
def message_cnt(soup):
    for x in soup.select('.head-item .push-tag'):
        tp = x.text[-1] 
        cont = int(x.text[:-1])
        if '推' == tp:
            push_cnt = cont
        elif '噓' == tp:
            boo_cnt = cont
        elif '→' == tp:
            neutral_cnt = cont

    count_cnt = push_cnt - boo_cnt
    all_cnt = push_cnt + boo_cnt + neutral_cnt

    message_count = {
          'all' : all_cnt
        , 'boo' : boo_cnt
        , 'count' : count_cnt
        , 'neutral' : neutral_cnt
        , 'push' : push_cnt        
    }
    return message_count

# get personal historical links

In [10]:
#usr_lst.remove('Neptunium')

# check what if only one page

In [11]:
invalid_rec = []

In [12]:
test_lst = [
      'yuxds'
, 'omanorbo'
, 'kieranc'
, 'mark2165'
, 'peter080808'
, 'linhu8883324'
, 'ebsd'
, 'idcc'
, 'OusakaMegumu'
, 'kim'
, 'kaminari22tw'
, 'a0986188522'
, 'monopoliest'
, 'zalora'
, 'ptt80357'
, 'mureka'
, 'dxoxb'
, 'pled' 
, 'ebsd' 
, 'sofc' 
, 'saed' 
, 'exponential7' 
, 'sigmaaldrich'
, 'Hartmann'
, 'Wojnarowski'
]

In [13]:
test_lst.extend([x for x in usr_lst if x not in test_lst])

In [14]:
fn_lst = []

for x in os.listdir(path):
    if x.endswith('id_check.json'):
        fn_lst.append(x.split('_')[0])

In [15]:
gogoog_lst = [x for x in test_lst if x not in fn_lst]
#invalid_rec = []

In [16]:
[x for x in usr_lst[:20] if x not in test_lst]

['Neptunium',
 'kero2377',
 'nicholas0406',
 'Kevin1103',
 'diefishfish',
 'KingKingCold',
 'hong888',
 'papamama007',
 'jobli',
 'ray90910',
 'ben108472',
 'ice80712',
 'AskWhat',
 'jason748',
 'oftisa']

In [16]:
len([x for x in test_lst if x not in fn_lst])

42200

In [17]:
gc.collect()

426

In [21]:
#pd.DataFrame(invalid_rec, columns=['usr_id', 'url', 'text', 'err_ms']).to_excel('invalid_rec_20191213.xlsx')

In [18]:
len(gogoog_lst)

42259

In [19]:
gc.collect()

0

In [28]:
#test_lst = ['KouChouChing']
#BrownNoser
#arsonlolita 
#tina81huang

In [29]:
#get all post ranking https://pttweb.tw/user/
#for user in usr_lst:
for user in test_lst:
    url = 'https://pttweb.tw/user/'
    print('user {} starts!'.format(user))
    url_lst = []
    rs_lst = []
    rs = requests.get(url + user, params={'page':1})
    soup = BeautifulSoup(rs.text)
    
    try:
        #check what if only one page
        lst_p = int(soup.findAll('a', {'rel':'nofollow'}, string='尾頁')[0]['href'].split('?page=')[1])
    except Exception as e:
        print(e)
        print('no data')
        invalid_rec.append([user, 'no_url', 'no_text', 'no_err'])
        continue
        
    print('Total {} pages'.format(lst_p))

    for page in range(1, lst_p + 1):    
        rs = requests.get(url + user, params={'page':page})
        soup = BeautifulSoup(rs.text)      
        #post urls
        for x in soup.select('.thread-title a'):
            url_lst.append('https://pttweb.tw' + x['href'])
        if page % 9 == 1:
            print(page)
    #get personal fully publish history
    for idx, url in enumerate(url_lst):
        messages = []
        remove_tags = [[True, {'class':['push', 'f2', 'f6']}], ['script']]
        try:
            rs = requests.get(url)
        except Exception as e:
            invalid_rec.append([user, url, x.text, e])
            print(idx)
            print(url)
            print(x.text)
            
        soup = BeautifulSoup(rs.text)

        #reply content
        for x in soup.select('.push'):
            try:
                messages.append({
                      'push_tag' : x.select('.push-tag')[0].text[0]
                    , 'push_userid' : x.select('.push-userid')[0].text
                    , 'push_content' : x.select('.push-content')[0].text[2:]
                    , 'push_ipdatetime' : x.select('.push-ipdatetime')[0].text.strip()
                })
            except Exception as e:
                invalid_rec.append([user, url, x.text, e])
                print(idx)
                print(url)
                print(x.text)

        try:
            art = content_parser(soup)        
            if art == 'NoContent':
                continue    
            message_aggr = message_cnt(soup)
            art.update({'message_count':message_aggr, 'messages':messages, 'author':user})
            temp_df = pd.DataFrame([art])
            rs_lst.append(temp_df)
        except Exception as e:
            invalid_rec.append([user, url, x.text, e])
            print(idx)
            print(e)
            print(url)  
        if idx % 300 == 1:
            print(idx)
            print(datetime.now())
    #output
    if rs_lst == []:
        continue
    fn_df = pd.concat(rs_lst)
    fn_df.reset_index(inplace=True)
    fn_df.to_json(path + '{}_id_check.json'.format(user))
    gc.collect()

user BrownNoser starts!
Total 54 pages
1
10
19
28
37
46
1
2019-12-15 12:44:47.901655
301
2019-12-15 12:50:03.368230
901
2019-12-15 13:00:06.421965


In [87]:
url_lst[0]

'https://pttweb.tw/s/40kcA'

# beerduck test

In [22]:
user = 'beerduck'

url = 'https://pttweb.tw/user/'
print('user {} starts!'.format(user))
url_lst = []
rs_lst = []
rs = requests.get(url + user, params={'page':1})
soup = BeautifulSoup(rs.text)

lst_p = int(soup.findAll('a', {'rel':'nofollow'}, string='尾頁')[0]['href'].split('?page=')[1])
print(lst_p)

user beerduck starts!
5


In [23]:
soup

<!DOCTYPE html>
<html lang="zh-hant" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<head>
<meta charset="utf-8"/>
<title>beerduck (beer鴨)發表的文章 | PTT Web</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="beerduck總共發表了97篇文章，最近發表的文章:[爆卦]夾手藍蝦官方粉絲團成立啦！（近況更新）12/10,[爆卦]藍蝦自夾手痛苦倒地影片爆紅，外國妹震驚12/09,[爆卦]網軍指揮徐弘庭和指傷陳玉珍都關臉書了！12/08,[爆卦]佛山爆發森林大火影片曝光12/07,Re:[問卦]狗狗在度估要叫他起來嗎12/07,[新聞]強推執勤女警藍委陳宜民函送究辦12/07" name="description"/>
<meta content="index,follow" name="robots"/>
<link href="https://pttweb.tw/user/beerduck/" rel="canonical"/>
<meta content="beerduck (beer鴨)發表的文章" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="2019-12-10T04:26:29+08:00" property="og:updated_time"/>
<meta content="https://pttweb.tw/user/beerduck/" property="og:url"/>
<meta content="beerduck總共發表了97篇文章，最近發表的文章:[爆卦]夾手藍蝦官方粉絲團成立啦！（近況更新）12/10,[爆卦]藍蝦自夾手痛苦倒地影片爆紅，外國妹

In [91]:
#'https://pttweb.twhttps://www.ptt.cc/bbs/tabletennis/M.1373104111.A.70F.html'
#url_lst[1713:1720]

['https://pttweb.tw/s/31WJ7',
 'https://pttweb.twhttps://www.ptt.cc/bbs/tabletennis/M.1373104111.A.70F.html',
 'https://pttweb.tw/s/31V6d',
 'https://pttweb.tw/s/31V5k',
 'https://pttweb.tw/s/31V2r',
 'https://pttweb.tw/s/31UVa',
 'https://pttweb.tw/s/31TbB']

In [84]:
idx

1714

In [14]:
#one no reply record
#fn_df.to_json(path + '{}_id_check.json'.format(user))

In [89]:
len(invalid_rec)

343

In [15]:
pd.read_csv('invalid_20191204.csv')

FileNotFoundError: [Errno 2] File b'invalid_20191204.csv' does not exist: b'invalid_20191204.csv'

In [18]:
#pd.DataFrame(invlid_rec).replace(',', '，').to_csv('invalid_20191204.csv')

,0,1,2,3
0,yuxds,https://pttweb.tw/s/2qb7l,990F檔案過大！部分文章無法顯示,list index out of range
1,omanorbo,no_url,no_text,no_err
2,kieranc,no_url,no_text,no_err
3,mark2165,https://pttweb.tw/s/3X1Br,56F→ nnkj: 不是警告而已嗎 1.171.165.24 10/15 19:36\n,More than one url....
4,mark2165,https://pttweb.tw/s/3T7eq,1F推 qooqoo2021: 不認識 IP看起來也沒有重複 09/21 16:23\n,local variable 'ori_url_split' referenced befo...
...,...,...,...,...
8566,wizardfizban,https://pttweb.tw/s/1NSTX,57F推 yasakihayate:還以為直接捅了!!? 03/25 14:44\n,local variable 'ori_url_split' referenced befo...
8567,wizardfizban,https://pttweb.tw/s/1LWFd,17F推 gfdsa:更正，回合制或者是wow like? 03/25 19:59\n,local variable 'ori_url_split' referenced befo...
8568,wizardfizban,https://pttweb.tw/s/1L9UA,28F推 pulsar1:小時候誰沒偷看過A片 跟長大變性犯罪者還是兩回事 03/18 13...,local variable 'ori_url_split' referenced befo...
8569,wizardfizban,https://pttweb.tw/s/1Kvuq,4F推 supermars:了解~~ 03/16 14:07\n,local variable 'ori_url_split' referenced befo...


# some unknown rec should save by just date and no year

In [11]:
url

'https://pttweb.tw/s/10QWn'

In [3]:
url_cnt = 0

#ori
#url = 'https://pttweb.tw/s/3n4eR'
#<span class="f2">※ 文章網址: <a href="https://www.ptt.cc/bbs/C_Chat/M.1575366675.A.20B.html" rel="nofollow" target="_blank">https://www.ptt.cc/bbs/C_Chat/M.1575366675.A.20B.html</a>
#</span>

#2017
#url = 'https://pttweb.tw/s/2LP5Y'

#hist
#url = 'https://pttweb.tw/s/10QWn'
#url = 'https://pttweb.tw/s/1yXfV'
#url = 'https://pttweb.tw/s/1tvdY'
#url = 'https://pttweb.tw/s/1caNp'
#url = 'https://pttweb.tw/s/2qb7l'
#url = 'https://pttweb.tw/s/31n9Q'

url = 'https://pttweb.tw/s/3JMji'

rs = requests.get(url)
soup = BeautifulSoup(rs.text)

ori_url_split = []
print(soup.select('.board')[0].text.strip())

ori_url_split = ''
ip = ''

for x in soup.select('span.f2'):
    print(x)
    print(x.text)
    print('*'*10)
    
    if x.text.startswith('※ 文章網址: https://www.ptt.cc/bbs'):
        ori_url_split = x.a['href'].split('/')
        ip = x.previous_element.split(':')[-1].split()[0]
        url_cnt = url_cnt + 1    
    elif x.text.startswith('※ 文章網址:'):
        pre_ip = [x for x in x.previous_elements][1]
        pre_url = x.next_sibling
        
        if type(pre_url) is bs4.element.NavigableString:
            #old article
            ori_url_split = x.a['href'].split('/')
        else:
            ori_url_split = x.next_sibling.text.split('/')
        
        if type(pre_ip) is bs4.element.NavigableString:
            ip = pre_ip.split(':')[-1].split()[0]
        else:
            #old article
            ip = pre_ip.text.split(':')[-1].split()[0]
            
            
        url_cnt = url_cnt + 1
    if url_cnt > 1:
        raise ValueError('More than one url....')
        
print(ori_url_split)

Hatepolitics
<span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 36.236.144.21
</span>
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 36.236.144.21

**********
<span class="f2">※ 文章網址: <a href="https://www.ptt.cc/bbs/HatePolitics/M.1555123536.A.947.html" rel="nofollow" target="_blank">https://www.ptt.cc/bbs/HatePolitics/M.1555123536.A.947.html</a>
</span>
※ 文章網址: https://www.ptt.cc/bbs/HatePolitics/M.1555123536.A.947.html

**********
<span class="f2">※ 編輯: geordie (36.236.144.21), 04/13/2019 10:53:07
</span>
※ 編輯: geordie (36.236.144.21), 04/13/2019 10:53:07

**********
<span class="f2">※ 編輯: geordie (36.236.144.21), 04/13/2019 10:56:14
</span>
※ 編輯: geordie (36.236.144.21), 04/13/2019 10:56:14

**********
['https:', '', 'www.ptt.cc', 'bbs', 'HatePolitics', 'M.1555123536.A.947.html']


In [11]:
x.next_sibling

In [37]:
type([x for x in x.previous_elements][1]) 
#is bs4.element.NavigableString

bs4.element.Tag

In [31]:
soup.select('.thread-posttime')

[]

In [15]:
url

'https://pttweb.tw/s/3n4eR'

In [11]:
#soup.findAll('a', {'rel':'nofollow'}, string='尾頁')#[0]['href']#.split('?page=')

[]

# url parsing full test

In [8]:
#url_lst = ['https://pttweb.tw/s/3XsPy']

#
#url_lst = ['https://pttweb.tw/s/3JMji']

invalid_rec = []
user = 'testing'
rs_lst = ['beer_duck']

In [12]:
for idx, url in enumerate(url_lst):
    messages = []
    remove_tags = [[True, {'class':['push', 'f2', 'f6']}], ['script']]
    rs = requests.get(url)
    soup = BeautifulSoup(rs.text)

    #reply content
    for x in soup.select('.push'):
        try:
            messages.append({
                  'push_tag' : x.select('.push-tag')[0].text[0]
                , 'push_userid' : x.select('.push-userid')[0].text
                , 'push_content' : x.select('.push-content')[0].text[2:]
                , 'push_ipdatetime' : x.select('.push-ipdatetime')[0].text.strip()
            })
        except Exception as e:
            invalid_rec.append([user, url, x.text, e])
            print('message parsing error')
            print(idx)
            print(url)
            print(x.text)

    try:
        art = content_parser(soup)        
        if art == 'NoContent':
            continue    
        message_aggr = message_cnt(soup)
        art.update({'message_count':message_aggr, 'messages':messages, 'author':user})
        temp_df = pd.DataFrame([art])
        rs_lst.append(temp_df)
    except Exception as e:
        print('art parsing error')
        invalid_rec.append([user, url, x.text, e])
        print(idx)
        print(e)
        print(url)

art parsing error
0
list index out of range
https://pttweb.tw/s/3JMji


In [18]:
soup.select('.post-time')
#.text.strip()
#art = content_parser(soup)

[]

In [23]:
len(pd.concat(rs_lst).messages.values[0])

1209

# get personal fully publish history

In [113]:
# for idx, url in enumerate(url_lst):
#     messages = []
#     remove_tags = [[True, {'class':['push', 'f2', 'f6']}], ['script']]
#     rs = requests.get(url)
#     soup = BeautifulSoup(rs.text)

#     #reply content
#     for x in soup.select('.push'):
#         try:
#             messages.append({
#                   'push_tag' : x.select('.push-tag')[0].text[0]
#                 , 'push_userid' : x.select('.push-userid')[0].text
#                 , 'push_content' : x.select('.push-content')[0].text[2:]
#                 , 'push_ipdatetime' : x.select('.push-ipdatetime')[0].text.strip()
#             })
#         except:
#             print(idx)
#             print(url)
#             print(x.text)
    
#     try:
#         art = content_parser(soup)        
#         if art == 'NoContent':
#             continue    
#         message_aggr = message_cnt(soup)
#         art.update({'message_count':message_aggr, 'messages':messages, 'author':user})
#         temp_df = pd.DataFrame([art])
#         rs_lst.append(temp_df)
#     except Exception as e:
#         print(idx)
#         print(e)
#         print(url)  
#     if idx % 300 == 1:
#         print(idx)
#         print(datetime.now())

1
36
More than one url....
https://pttweb.tw/s/3X1Br
51
local variable 'ori_url_split' referenced before assignment
https://pttweb.tw/s/3T7eq
52
More than one url....
https://pttweb.tw/s/3T7ep
62
local variable 'ori_url_split' referenced before assignment
https://pttweb.tw/s/3SNRN
91
local variable 'ori_url_split' referenced before assignment
https://pttweb.tw/s/3Op3R
101
121
local variable 'ori_url_split' referenced before assignment
https://pttweb.tw/s/3M5jI
201
301
401
501
601
701
801
901
1001


In [8]:
#test re
#v url = 'https://pttweb.tw/s/3a6AF' #re with ori
#v url = 'https://pttweb.tw/s/3XuNB' #re without ori
#v url = 'https://pttweb.tw/s/3aYJ9' #colorful content
#v url = 'https://pttweb.tw/s/3Yk0l' #simple news
#url = 'https://pttweb.tw/s/3Yjaj' #forward

In [0]:
#M.1119222611.A.7A9
# {
#     "article_id": 文章 ID,
#     "article_title": 文章標題 ,
#     "author": 作者,
#     "board": 板名,
#     "content": 文章內容,
#     "date": 發文時間,
#     "ip": 發文位址,
#     "message_count": { # 推文
#         "all": 總數,
#         "boo": 噓文數,
#         "count": 推文數-噓文數,
#         "neutral": → 數,
#         "push": 推文數
#     },
#     "messages": [ # 推文內容
#       {
#         "push_content": 推文內容,
#         "push_ipdatetime": 推文時間及位址,
#         "push_tag": 推/噓/→ ,
#         "push_userid": 推文者 ID
#       },
#       ...
#       ]
# }

SyntaxError: ignored

In [0]:
#轉流量
#https://www.ptt.cc/bbs/Gossiping/M.1574431993.A.6DE.html
#certain media's forward
#https://pttweb.tw/s/3a6AF
#ws([t1.text])